In [1]:
# =========================
# END-TO-END: SPY + VIX DATA + HEDGE LEVELS
# =========================

import math
import numpy as np
import pandas as pd


In [ ]:

# ------------- CONFIG -------------
START_DATE = None          # e.g. "2025-11-01" or None to use PERIOD
PERIOD = "6mo"            # used if START_DATE is None; e.g. "3mo", "6mo", "1y"
VOL_LOOKBACK = 15         # rolling window for avg_vol
HORIZON_DAYS = 15         # your horizon for sqrt(15)
SIGMA_MULT = 2.0          # "2 STD"
SPY_STRIKE_STEP = 1.0     # strike rounding step for SPY (often 1.0)
VIX_STRIKE_STEP = 0.5     # strike rounding step for VIX (often 0.5)
USE_ADJ_CLOSE_FOR_RET = False  # returns computed from close by default (set True to use adj close)

# ------------- HELPERS -------------
def round_down(x, step):
    return math.floor(x / step) * step

def round_up(x, step):
    return math.ceil(x / step) * step

def fetch_with_yfinance(symbol: str, period: str = "6mo", start: str = None) -> pd.DataFrame:
    """
    Fetch daily data using yfinance.
    Returns dataframe with columns: Open, High, Low, Close, Adj Close (if available), Volume
    Index: DatetimeIndex
    """
    try:
        import yfinance as yf
    except ImportError as e:
        raise ImportError(
            "yfinance is not installed. Install it with:\n"
            "pip install yfinance"
        ) from e

    if start:
        df = yf.download(symbol, start=start, interval="1d", auto_adjust=False, progress=False)
    else:
        df = yf.download(symbol, period=period, interval="1d", auto_adjust=False, progress=False)

    if df is None or df.empty:
        raise ValueError(f"No data returned for {symbol}. Check symbol or network.")

    # Standardize column names
    df = df.rename(columns={
        "Open": "open",
        "High": "high",
        "Low": "low",
        "Close": "close",
        "Adj Close": "adj_close",
        "Volume": "volume",
    })

    # Some yfinance versions return multiindex columns; flatten if needed
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [c[0] if isinstance(c, tuple) else c for c in df.columns]

    df = df.dropna(subset=["open", "close"])
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    return df

def compute_levels(spy_df: pd.DataFrame, vix_df: pd.DataFrame) -> pd.DataFrame:
    df = spy_df.copy()

    # 1) SPY 3-day average (close-based)
    df["spy_avg"] = df["close"].rolling(3).mean()

    # 2) avg_vol (daily) using rolling std of log returns
    px = df["adj_close"] if (USE_ADJ_CLOSE_FOR_RET and "adj_close" in df.columns) else df["close"]
    df["ret"] = np.log(px / px.shift(1))
    df["avg_vol"] = df["ret"].rolling(VOL_LOOKBACK).std()

    sqrt_h = np.sqrt(HORIZON_DAYS) 
    shock = SIGMA_MULT * df["avg_vol"] * sqrt_h  # this is a % move (if avg_vol is daily return vol)

    # 3) SPY PUT strike (unit-consistent)
    df["spy_put_strike_raw"] = df["open"] * (1.0 - shock)

    # Merge VIX spot (close)
    vix = vix_df[["close"]].rename(columns={"close": "vix_spot"}).copy()
    out = df.merge(vix, left_index=True, right_index=True, how="left")

    # 4) VIX CALL strike — Version A: your direct spec (NOT unit-consistent, but matches your statement)
    out["vix_call_strike_raw_direct"] = out["vix_spot"] + (SIGMA_MULT * out["avg_vol"] * sqrt_h)

    # 4) VIX CALL strike — Version B (recommended): convert SPY daily vol to a VIX-like scale
    # Convert SPY daily vol to annualized percent, then apply 15-day scaling back
    out["spy_vol_ann_pct"] = out["avg_vol"] * np.sqrt(252) * 100.0
    out["vix_call_strike_raw_converted"] = out["vix_spot"] + (SIGMA_MULT * out["spy_vol_ann_pct"] * np.sqrt(HORIZON_DAYS / 252.0))

    # Rounding strikes to tradable increments
    out["spy_put_strike"] = out["spy_put_strike_raw"].apply(lambda x: round_down(x, SPY_STRIKE_STEP) if pd.notna(x) else np.nan)

    # For calls, round UP
    out["vix_call_strike_direct"] = out["vix_call_strike_raw_direct"].apply(lambda x: round_up(x, VIX_STRIKE_STEP) if pd.notna(x) else np.nan)
    out["vix_call_strike_converted"] = out["vix_call_strike_raw_converted"].apply(lambda x: round_up(x, VIX_STRIKE_STEP) if pd.notna(x) else np.nan)

    # Clean output columns
    keep = [
        "open", "close", "spy_avg", "avg_vol",
        "spy_put_strike",
        "vix_spot",
        "vix_call_strike_direct",
        "vix_call_strike_converted",
    ]
    out = out[keep].copy()

    # Add a friendly date column too
    out = out.reset_index().rename(columns={"index": "date"})
    return out

# ------------- RUN -------------
if __name__ == "__main__":

    # Fetch data
    spy = fetch_with_yfinance("SPY", period=PERIOD, start=START_DATE)
    vix = fetch_with_yfinance("^VIX", period=PERIOD, start=START_DATE)

    results = compute_levels(spy, vix)

    # Show last few rows
    pd.set_option("display.width", 140)
    pd.set_option("display.max_columns", 50)
    print(results.tail(15))

    # Optional: save
    out_file = "spy_vix_hedge_levels.csv"
    results.to_csv(out_file, index=False)
    print(f"\nSaved: {out_file}")


          Date        open       close     spy_avg   avg_vol  spy_put_strike   vix_spot  vix_call_strike_direct  vix_call_strike_converted
112 2026-01-13  695.489990  693.770020  694.333333  0.004307           672.0  15.980000                    16.5                       19.5
113 2026-01-14  691.000000  690.359985  693.096659  0.004355           667.0  16.750000                    17.0                       20.5
114 2026-01-15  694.570007  692.239990  692.123332  0.004259           671.0  15.840000                    16.0                       19.5
115 2026-01-16  693.659973  691.659973  691.419983  0.004180           671.0  15.860000                    16.0                       19.5
116 2026-01-20  681.489990  677.580017  687.159993  0.006786           645.0  20.090000                    20.5                       25.5
117 2026-01-21  679.650024  685.400024  684.880005  0.007492           640.0  16.900000                    17.0                       23.0
118 2026-01-22  689.849976 